#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: May 5th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [0]:
!pip install -q gpt_2_simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## Verify GPU

Colaboratory now uses an Nvidia T4 GPU, which is slightly faster than the old Nvidia K80 GPU for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text. However sometimes the K80 will still be used.

You can verify which GPU is active by running the cell below.

In [0]:
!nvidia-smi

Tue May  7 04:49:45 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are two sizes of GPT-2:

* `117M` (default): the "small" model, 500MB on disk.
* `345M`: the "medium" model, 1.5GB on disk.

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [0]:
gpt2.download_gpt2(model_name="345M")

Fetching checkpoint: 1.00kit [00:00, 252kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 45.6Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 318kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:25, 54.6Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 2.64Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 39.4Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 31.2Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [0]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
file_name = "dril.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              learning_rate=1e-5,
              dataset=file_name,
              model_name='345M',
              steps=1000,
              restore_from='fresh',
              print_every=10,
              sample_every=50,
            save_every=100
              )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Loading checkpoint models/345M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/345M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.42it/s]


dataset has 64874 tokens
Training...
[10 | 25.18] loss=4.21 avg=4.21
[20 | 40.83] loss=4.29 avg=4.25
[30 | 56.98] loss=3.99 avg=4.16
[40 | 73.76] loss=4.03 avg=4.13
[50 | 91.12] loss=3.80 avg=4.06
======== SAMPLE 1 ========
P. and B. A. [1999] 2 S.R. 859.

[11] For detailed accounts see: (a) "A Guide to the Laws of Criminal Procedure (England and Wales)", by Mr Michael Wylie at a meeting of the Association of British Criminal Jurists on May 13, 2007, at www.law.ac.uk/hdl/pp3138/mw

(b) C&D. (BMI)

[12] The Law Commission of the UK and the Law Society of the City (LSE) have published guidelines for the application of the Crown's powers under Chapter 4 of the Crown's Code and Regulations.

[13] The Law Commission of the UK and The LSE have prepared a booklet called, "The Crown's Code: A Guide Through the Law," which can be downloaded from www.law.ac.uk/mso/cbs/codes/cbsguide.htm

[14] At www.law.ac.uk/rules_general/cbs_guide_1.pdf

[15] At http://www.courtroom.ac.uk/p/p_principal/P_princ

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. (NB: if you are downloading the model to your personal computer, download the large model checkpoint file *seperately*, download the other files, and reconstruct the `/checkpoint/run1` folder hierarchy locally).

In [0]:
gpt2.copy_checkpoint_to_gdrive()

FileExistsError: ignored

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `checkpoint` folder from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive()

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
gpt2.generate(sess)


The most powerful weapon a man can wield is his brain. -Brian Eno


"I find it more interesting to be right than to be right-ish" -Hugh Everett


"The people you should be killing right now are the people who don't read the news" -Matt Taibbi


"In my book, those two things are the same thing" -Donald X. Vaccarino (via email)


"I find it very hard to believe that I am not being personally attacked for posting something like "5 Minute Masturbation" on my blog" -Gabe Newell


"I find it absurd that every time I post a new review it is Impersonated as "A Man's Day" by a mob" -Curtis Fimmel


"I find it utterly ludicrous that the average joe who sends me a message wants to hear, "How to clean a toilet" over and over again, because that is exactly how I feel" -Dan Aykroyd


"I find it absurd that every time I post a new review it is Impersonated as "A Man's Day" by a mob" -Dan Aykroyd


"I find it absurd that every time I post a new review it is Impersonated as " a man " on the internet, 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
gpt2.generate(sess,
              length=250,
              temperature=1,
              prefix="SEP ",
              nsamples=7000,
              batch_size=5
              )

SP ive had enough of Americans peacefully expressing shock and disgust at news of terrorism. 1" is enough. ou sahib king man... SEP Bessie Rivera is one of the heroes of the hit CBS sitcom "The Big Bang Theory." SEP im very close to getting 10 years. im banned from all of school for it SEP doing them jigsomes hands like a girl\n31017: Standing on end of toothbrush in Nordstrom parking lot\n69329: Standing in line at H&M\n78866: in a toilet at Saks Fifth Avenue\n SEP I BLAME EVERY BUNCH OF MEN <<< before Pissed SEP AH! You just pushed the maximum ur tolerance to 11 times its pure alkaline level. The screen goes back to normal. Your dick shrinks 10cc0000%. SEP mayhem (heading the dogfight between john and robinson need i say more ?)) enjoys agitating his users SEP they say a bad review stings, but this hurts more than a bad car SEP man who sell\'s speakers to apartment buildings for $100 a pop, in latest video posted to instagram explaining his despicable actions SEP (trying to board

SP

KeyboardInterrupt: ignored

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp and then download it.

You can rerun the cell as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=1,
                      nsamples=100,
                      batch_size=20
                      )

files.download(gen_file)

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail or out-of-memory/OOM), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.